# LSTM Model


## Import Data
### Players from RPS_game.py

In [1]:
from Lib.RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from Lib.RPS_encoding import RPSpair_encode, RPSpair_decode, RPSpair_encode_series
from Lib.Telepathic_player import telepath
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, LSTM, Embedding
import pandas as pd
from os import listdir
from os.path import isfile, join

## Read and Concatenate Games
Take all games and load into dataframe

In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join

path = "Games/"
all_game_paths = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
games_concat = pd.DataFrame(columns = ["Player","Opponent","Result","Winning_Play"])
games = []
for game_path in all_game_paths:
    #print(game_path)
    game_data = pd.read_csv(game_path, names = ["Player","Opponent","Result","Winning_Play"])
    games.append(game_data)
    games_concat = games_concat.append(game_data)
    
#print(len(game_data.index)) 
#game_data.head()  
print(len(games_concat.index))
games_concat.head()

100000


,Player,Opponent,Result,Winning_Play
0,S,S,0,R
1,R,P,0,S
2,R,R,0,P
3,P,R,1,P
4,R,P,0,S


# LSTM Training
### Parameters

In [3]:
seq_length = 100 #how far back the ltsm looks when training
#Model parameters
BATCH_SIZE = 64
VOCAB_SIZE = 9 # RPS pairs
EMBEDDING_DIM = 32
RNN_UNITS = 1024

TRAINING_EPOCHS = 100

#Kris_Mrugesh settings
#seq_length = 100 
#BATCH_SIZE = 64
#VOCAB_SIZE = 3 
#EMBEDDING_DIM = 128
#RNN_UNITS = 512
#TRAINING_EPOCHS = 300


### Format data for LTSM training

In [4]:
#Encoding scheme:
#RR = 0
#RP = 1
#RS = 2
#PR = 3
#PP = 4
#PS = 5
#SR = 6
#SP = 7
#SS = 8

temp = games_concat.drop("Result", axis = 1)
temp.index = [x for x in range(temp.index.size)]
tempInput = pd.Series([temp.loc[x,"Player"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])
tempTarget = pd.Series([temp.loc[x,"Winning_Play"] + temp.loc[x,"Opponent"] for x in range(temp['Player'].size)])

#Convert letter pairs to categorical integers
tempInput = RPSpair_encode_series(tempInput)
tempTarget = RPSpair_encode_series(tempTarget)

In [5]:
input_data = tf.data.Dataset.from_tensor_slices(tempInput)
target_data = tf.data.Dataset.from_tensor_slices(tempTarget)

#split into sequences
input_sequences = input_data.batch(seq_length+1, drop_remainder = True)
target_sequences = target_data.batch(seq_length+1, drop_remainder = True)
print(input_sequences)

#remove last element from all input sequences
def remove_last(chunk):
    return chunk[:-1]
inputs = input_sequences.map(remove_last)

#remove first element from all target sequences
def remove_first(chunk):
    return chunk[1:]
targets = target_sequences.map(remove_first)

full_dataset = tf.data.Dataset.zip((inputs, targets)).shuffle(10000)

dataset_size = len(full_dataset)
train_size = int(0.7*dataset_size)
test_size = int(0.15*dataset_size)
val_size = int(0.15*dataset_size)

train_dataset = full_dataset.take(train_size).batch(BATCH_SIZE, drop_remainder = True)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.take(test_size).batch(BATCH_SIZE, drop_remainder = True)
test_dataset = test_dataset.skip(test_size).batch(BATCH_SIZE, drop_remainder = True)

print("Total sequence count: " + str(dataset_size))
print("Training set: " + str(len(train_dataset)))
print("Test set: " + str(len(test_dataset)))
print("Validation set: " + str(len(val_dataset)))
for element in train_dataset.take(1):
    print(element)


<BatchDataset shapes: (101,), types: tf.int64>
Total sequence count: 990
Training set: 10
Test set: 2
Validation set: 2
(<tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[5, 2, 0, ..., 3, 5, 8],
       [2, 7, 7, ..., 3, 2, 2],
       [3, 7, 2, ..., 7, 7, 3],
       ...,
       [3, 5, 8, ..., 4, 8, 6],
       [0, 8, 3, ..., 8, 3, 8],
       [2, 2, 7, ..., 3, 3, 2]], dtype=int64)>, <tf.Tensor: shape=(64, 100), dtype=int64, numpy=
array([[2, 3, 3, ..., 2, 2, 3],
       [7, 7, 3, ..., 2, 2, 7],
       [7, 2, 3, ..., 7, 3, 7],
       ...,
       [2, 2, 3, ..., 2, 3, 3],
       [2, 3, 2, ..., 3, 2, 3],
       [2, 7, 7, ..., 3, 2, 2]], dtype=int64)>)


### Build the model

In [7]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential()
    model.add(Embedding(vocab_size, embedding_dim, batch_input_shape = [batch_size, None]))
    model.add(LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size+1))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 32)            288       
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4329472   
_________________________________________________________________
dense (Dense)                (64, None, 10)            10250     
Total params: 4,340,010
Trainable params: 4,340,010
Non-trainable params: 0
_________________________________________________________________


In [6]:
#OR LOAD MODEL FROM SAVE
model = keras.models.load_model('Models/LSTM_BalancedTraining_GameHistoryBased_Quincy_Kris_Mrugesh')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 32)             288       
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           4329472   
_________________________________________________________________
dense_1 (Dense)              (1, None, 10)             10250     
Total params: 4,340,010
Trainable params: 4,340,010
Non-trainable params: 0
_________________________________________________________________


### Create loss function

In [8]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits = True)

### Compile the model

In [9]:
model.compile(optimizer='adam', loss=loss)

### Create Checkpoints
configure the model to create checkpoints as it trains so the model can be reloaded and continue training

In [10]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix, save_weights_only = True)

print(tf.__version__)

2.3.1


In [11]:
#CALL TO LOAD LATEST CHECKPOINT FOR TRAINING
checkpoint_dir = './training_checkpoints'
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

### Training the model

In [12]:
history = model.fit(train_dataset, epochs = TRAINING_EPOCHS, callbacks = [checkpoint_callback], validation_data=val_dataset)
model.evaluate(test_dataset, batch_size=128) 

Epoch 1/100
10/10 [==============================] - 1s 87ms/step - loss: 1.8675 - val_loss: 1.6369
Epoch 2/100
10/10 [==============================] - 0s 49ms/step - loss: 1.2722 - val_loss: 1.1187
Epoch 3/100
10/10 [==============================] - 0s 48ms/step - loss: 1.1301 - val_loss: 1.1054
Epoch 4/100
10/10 [==============================] - 0s 48ms/step - loss: 1.0795 - val_loss: 1.0718
Epoch 5/100
10/10 [==============================] - 0s 50ms/step - loss: 1.0422 - val_loss: 1.0301
Epoch 6/100
10/10 [==============================] - 0s 50ms/step - loss: 1.0201 - val_loss: 1.0342
Epoch 7/100
10/10 [==============================] - 0s 48ms/step - loss: 1.0019 - val_loss: 0.9772
Epoch 8/100
10/10 [==============================] - 1s 53ms/step - loss: 0.9711 - val_loss: 0.9597
Epoch 9/100
10/10 [==============================] - 1s 51ms/step - loss: 0.9540 - val_loss: 0.9441
Epoch 10/100
10/10 [==============================] - 0s 49ms/step - loss: 0.9335 - val_loss: 0.8883

Epoch 80/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0884 - val_loss: 0.0884
Epoch 81/100
10/10 [==============================] - 1s 51ms/step - loss: 0.0868 - val_loss: 0.0966
Epoch 82/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0866 - val_loss: 0.0879
Epoch 83/100
10/10 [==============================] - 0s 50ms/step - loss: 0.0881 - val_loss: 0.0923
Epoch 84/100
10/10 [==============================] - 0s 50ms/step - loss: 0.0841 - val_loss: 0.0818
Epoch 85/100
10/10 [==============================] - 0s 48ms/step - loss: 0.0876 - val_loss: 0.0697
Epoch 86/100
10/10 [==============================] - 1s 54ms/step - loss: 0.0860 - val_loss: 0.0848
Epoch 87/100
10/10 [==============================] - 1s 51ms/step - loss: 0.0803 - val_loss: 0.0817
Epoch 88/100
10/10 [==============================] - 1s 51ms/step - loss: 0.0830 - val_loss: 0.0842
Epoch 89/100
10/10 [==============================] - 1s 51ms/step - loss: 0.0800 - val_los

0.08352863043546677

In [13]:
history = model.fit(train_dataset, epochs = 10, callbacks = [checkpoint_callback], validation_data=val_dataset)
model.evaluate(test_dataset, batch_size=128) 

Epoch 1/10
10/10 [==============================] - 1s 54ms/step - loss: 0.0723 - val_loss: 0.0772
Epoch 2/10
10/10 [==============================] - 1s 50ms/step - loss: 0.0726 - val_loss: 0.0818
Epoch 3/10
10/10 [==============================] - 0s 50ms/step - loss: 0.0746 - val_loss: 0.0923
Epoch 4/10
10/10 [==============================] - 1s 52ms/step - loss: 0.0712 - val_loss: 0.0674
Epoch 5/10
10/10 [==============================] - 1s 52ms/step - loss: 0.0737 - val_loss: 0.0822
Epoch 6/10
10/10 [==============================] - 0s 48ms/step - loss: 0.0731 - val_loss: 0.0734
Epoch 7/10
10/10 [==============================] - 0s 49ms/step - loss: 0.0702 - val_loss: 0.0786
Epoch 8/10
10/10 [==============================] - 0s 50ms/step - loss: 0.0725 - val_loss: 0.0748
Epoch 9/10
10/10 [==============================] - 0s 49ms/step - loss: 0.0746 - val_loss: 0.0706
Epoch 10/10
2/2 [==============================] - 0s 9ms/step - loss: 0.0677


0.0676712840795517

### Rebuild model for predicting one output rather than a vector

In [14]:
#change config
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size = 1)
#load trained weights from checkpoint
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generate Outputs from Model

In [1]:
def Get_Prediction(model, opponent_history, player_history):
    input_raw = [x + y if x != '' and y != '' else 'PR' for (x,y) in zip(player_history, opponent_history)]
    #print(input_raw)
    #format history for input
    input_eval = [RPSpair_encode(c) for c in input_raw]
    input_eval = tf.expand_dims(input_eval, 0)
    
    #high temperature  => suprising output
    #low temperature => predictable output
    temperature = 0.1; 
    
    model.reset_states()
    
    prediction = model(input_eval)
    prediction = tf.squeeze(prediction, 0)
    
    prediction = prediction / temperature
    predicted_id = tf.random.categorical(prediction, num_samples = 1)[-1,0].numpy()
    #print(predicted_id)
    #print("Guessing: " + RPSpair_decode(predicted_id))
    return RPSpair_decode(predicted_id)[0]

#print(Get_Prediction(model, ['R','R','P']))

# RPS.py
Player function

In [8]:
plays = 1000

def player(prev_play, opponent_history = [], play_history=['']):
    if len(play_history) > plays: 
        opponent_history.clear()
        play_history.clear()
        play_history.append('')
    opponent_history.append(prev_play)
    #print("Actual: " + prev_play) 
    guess = Get_Prediction(model, opponent_history, play_history)
    #print(guess)
    play_history.append(guess)
    return guess

# MAIN.PY

In [2]:
play(player, quincy, plays)
play(player, abbey, plays)
play(player, kris, plays)
play(player, mrugesh, plays)

NameError: name 'play' is not defined

## Saving the model

In [18]:
model.save('Models/LSTM_BalancedTraining_GameHistoryBased_Quincy_Kris_Mrugesh')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/LSTM_BalancedTraining_GameHistoryBased_Quincy_Kris_Mrugesh\assets
